In [4]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, BertForTokenClassification
import torch
device = torch.device("cuda")

model_name = 'AbderrahmanSkiredj1/prefinetuned_arabertv02_then_finetuned_on_abbad_data'
tokenizer = AutoTokenizer.from_pretrained(model_name)



#test = load_dataset("AbderrahmanSkiredj1/fadel_test_sentences_sizeok_as_token_classif")
test_short = load_dataset('AbderrahmanSkiredj1/abbad_test_short_sentences')
test_long = load_dataset('AbderrahmanSkiredj1/abbad_test_long_sentences')

fadel_or_abbad = "abbad"
#print(test)

text_column_name = "tokens"
label_column_name = "labels"

if fadel_or_abbad == "fadel":
    b_to_i_label = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    label_to_id = {'X': -100, 'تطويل': 1, 'سكون': 2, 'شدة': 3, 'شدة ضمة': 4, 'شدة ضمتان': 5, 'شدة فتحة': 6, 'شدة فتحتان': 7, 'شدة كسرة': 8, 'شدة كسرتان': 9, 'ضمة': 10, 'ضمتان': 11, 'فتحة': 12, 'فتحتان': 13, 'كسرة': 14, 'كسرتان': 15}
    label_list = ['X', 'تطويل', 'سكون', 'شدة', 'شدة ضمة', 'شدة ضمتان', 'شدة فتحة', 'شدة فتحتان', 'شدة كسرة', 'شدة كسرتان', 'ضمة', 'ضمتان', 'فتحة', 'فتحتان', 'كسرة', 'كسرتان']
else:
    b_to_i_label = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
    label_to_id = {'X': -100, 'تطويل': 1, 'سكون': 2, 'شدة': 3, 'شدة سكون': 4, 'شدة شدة': 5, 'شدة ضمة': 6, 'شدة ضمتان': 7, 'شدة فتحة': 8, 'شدة فتحتان': 9, 'شدة كسرة': 10, 'شدة كسرتان': 11, 'ضمة': 12, 'ضمتان': 13, 'فتحة': 14, 'فتحتان': 15, 'كسرة': 16, 'كسرتان': 17}
    label_list = ['X', 'تطويل', 'سكون', 'شدة', 'شدة سكون', 'شدة شدة', 'شدة ضمة', 'شدة ضمتان', 'شدة فتحة', 'شدة فتحتان', 'شدة كسرة', 'شدة كسرتان', 'ضمة', 'ضمتان', 'فتحة', 'فتحتان', 'كسرة', 'كسرتان']

num_labels = len(label_list)
padding = "max_length"
padding = "longest"
padding=False
max_seq_length=512
label_all_tokens=False
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        #truncation=True,
        #max_length=max_seq_length,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                if label_all_tokens:
                    label_ids.append(b_to_i_label[label_to_id[label[word_idx]]])
                else:
                    label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

test_long = test_long.map(
    tokenize_and_align_labels,
    batched=True,
    #load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)
test_short = test_short.map(
    tokenize_and_align_labels,
    batched=True,
    #load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.76M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Generating train split:   0%|          | 0/149872 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/128 [00:00<?, ? examples/s]

Running tokenizer on train dataset:   0%|          | 0/128 [00:00<?, ? examples/s]

Running tokenizer on train dataset:   0%|          | 0/149872 [00:00<?, ? examples/s]

In [5]:
def compute_der_wer_numbers(zipped_list):
    # Reinitialize the list and variables with correct logic for handling consecutive -100 and accurate DER and WER calculation
    total_characters = 0
    incorrect_characters = 0
    total_words = 0
    incorrect_words = 0
    current_word_correct = True
    # This time, ensuring we only count a new word when we've actually seen diacritics following -100
    just_seen_100 = False  # Indicates we've just seen a -100, helps in handling consecutive -100s

    for gt, pred in zipped_list:
        if gt == -100:
            if not just_seen_100:  # Only consider starting a new word if we haven't just seen a -100
                just_seen_100 = True
                if not current_word_correct:  # If the last word (if any) was incorrect
                    incorrect_words += 1
                current_word_correct = True  # Assume the new word is correct until proven otherwise
        else:  # We are in a word with diacritics
            if just_seen_100:  # Transitioning from -100 to diacritics, mark a new word
                total_words += 1
                just_seen_100 = False  # Reset since we've now started a new word
            total_characters += 1
            if gt != pred:
                incorrect_characters += 1
                current_word_correct = False

    # After loop, if the last word was incorrect and it was a real word (not just a -100), account for it
    if not just_seen_100 and not current_word_correct:  # Check for the last word if it was incorrect
        incorrect_words += 1

    # Recalculate DER and WER with corrected logic
    #der = incorrect_characters / total_characters if total_characters > 0 else 0
    #wer = incorrect_words / total_words if total_words > 0 else 0
    return incorrect_characters, total_characters, incorrect_words, total_words

def compute_error_numbers_for_sample_original(input_ids, labels, model):
    inp = torch.tensor(input_ids)
    inp = inp.unsqueeze(0)
    inp = inp.to(device)
    o3 = model(inp)
    lg3 = o3.logits
    predicted_labels = torch.argmax(lg3, dim=-1)
    zi = list(zip(labels, predicted_labels.tolist()[0]))
    return compute_der_wer_numbers(zi)

def compute_error_numbers_for_sample(input_ids, labels, model, step=20, no_slide = False):
    if len(input_ids) < 512 or no_slide:
        inp = torch.tensor(input_ids)
        inp = inp.unsqueeze(0)
        inp = inp.to(device)
        o3 = model(inp)
        lg3 = o3.logits
        predicted_labels = torch.argmax(lg3, dim=-1)
        zi = list(zip(labels, predicted_labels.tolist()[0]))
        return compute_der_wer_numbers(zi)
    else:
        return compute_error_numbers_for_sample_with_slides(input_ids, labels, model, step)

In [6]:
def compute_error_numbers_for_sample_with_slides(input_ids, labels, model, step):
    sublists, input_id_mapping = get_windows_of_512_from_long_inputids(input_ids, labels, step)
    predictions_by_sublists = dict()
    for i, sub in enumerate(sublists):
        subinput_ids = sub['input_ids']+[3]
        sublabels = sub['labels']+[-100]
        inp = torch.tensor(subinput_ids)
        inp = inp.unsqueeze(0)
        inp = inp.to(device)
        o3 = model(inp)
        lg3 = o3.logits
        predicted_labels = torch.argmax(lg3, dim=-1)
        #subzi = list(zip(sublabels, predicted_labels.tolist()[0]))
        #all_zipped.append(subzi)
        predictions_by_sublists[i] = predicted_labels.tolist()[0]
    final_predictions = []
    for k in input_id_mapping:
        predicted_labels_for_k = []
        for sub in input_id_mapping[k]:
            indices_of_k_in_sub = input_id_mapping[k][sub]
            predicted_labels_for_k.append(predictions_by_sublists[sub][indices_of_k_in_sub])
        final_predictions.append(max(predicted_labels_for_k, key = predicted_labels_for_k.count))
    zi = list(zip(labels, final_predictions))
    return compute_der_wer_numbers(zi)

In [7]:
def get_windows_of_512_from_long_inputids(input_ids, labels, step):
    my_list = input_ids
    sublists = []
    input_id_mapping = {}
    start_index = 0
    sublist_index = 0  # Initialize a counter for the sublist index
    have_we_reached_end = False
    while start_index < len(my_list):
        temp_sum = 0
        temp_list = []
        i = start_index
        while i < len(my_list):
            is_next_one_a_four = False
            if temp_sum + 1 > 500:
                break
            j = i+1
            if j<len(my_list):
                if my_list[j]==4:
                    is_next_one_a_four = True
                    while j+1 < len(my_list):
                        if my_list[j+1]==4:
                            j += 1
                        else:
                            break
            #(i + j-i ==> +1 + j-i)
            if not is_next_one_a_four:
                temp_sum += 1
                temp_list.append(my_list[i])
                if i not in input_id_mapping:
                    #input_id_mapping[i] = [sublist_index]
                    input_id_mapping[i] = {sublist_index: len(temp_list) - 1}
                else:
                    #input_id_mapping[i].append(sublist_index)
                    input_id_mapping[i][sublist_index] = len(temp_list) - 1
                i += 1
            else:
                temp_sum += 1+j-i
                temp_list.extend(my_list[i:j+1])
                for k in range(i, j + 1):
                    if k not in input_id_mapping:
                        input_id_mapping[k] = {sublist_index: len(temp_list) - (j + 1 - k)}
                        #input_id_mapping[k] = [sublist_index]
                    else:
                        input_id_mapping[k][sublist_index] = len(temp_list) - (j + 1 - k)
                        #input_id_mapping[k].append(sublist_index)
                i = j+1
            if i>len(my_list)-1:
                have_we_reached_end = True

        sublabels = labels[start_index:start_index+len(temp_list)]
        sublists.append({"input_ids": temp_list, "labels": sublabels})
        sublist_index += 1

        start_index += step  # Slide by the step size

        # Ensure start_index does not point to a "4", and skip any "4"s if necessary
        while start_index < len(my_list) and my_list[start_index] == 4:
            start_index += 1  # Skip "4"


        # Make sure start_index does not exceed the list length after adjustment
        if start_index >= len(my_list):
            break  # Exit the loop if we have reached or exceeded the list length

        if have_we_reached_end:
            break
    return sublists, input_id_mapping

In [9]:
model_name = "AbderrahmanSkiredj1/arabertv02_prefinetuned_CAPostagSegm_then_ATDMLM_then_finetuned_tashkeel_abbad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
test_short = load_dataset('AbderrahmanSkiredj1/abbad_test_short_sentences')
test_long = load_dataset('AbderrahmanSkiredj1/abbad_test_long_sentences')
test_long_splitted = load_dataset('AbderrahmanSkiredj1/abbad_test_long_sentences_splitted')

test_long = test_long.map(
    tokenize_and_align_labels,
    batched=True,
    #load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)
test_short = test_short.map(
    tokenize_and_align_labels,
    batched=True,
    #load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)
test_long_splitted = test_long_splitted.map(
    tokenize_and_align_labels,
    batched=True,
    #load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)
test_long = test_long['train']
test_long_splitted = test_long_splitted['train']
test_short = test_short['train']

Running tokenizer on train dataset:   0%|          | 0/149872 [00:00<?, ? examples/s]

Running tokenizer on train dataset:   0%|          | 0/262 [00:00<?, ? examples/s]

In [10]:
device = torch.device("cuda")
model = BertForTokenClassification.from_pretrained(model_name)
model.to(device)

config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/539M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
from tqdm import tqdm
total_nb_words= 0
total_nb_characters = 0
incorrect_words=0
incorrect_characters=0
j=0
for sample in tqdm(test_short):
    delta_incorrect_characters, delta_total_characters, delta_incorrect_words, delta_total_words = compute_error_numbers_for_sample(sample['input_ids'], sample['labels'], model, step=10, no_slide=True)
    total_nb_words += delta_total_words
    total_nb_characters += delta_total_characters
    incorrect_words += delta_incorrect_words
    incorrect_characters += delta_incorrect_characters
    j += 1
    if j%3000==0:
      print(100*incorrect_characters/total_nb_characters, 100*incorrect_words/total_nb_words)

  2%|▏         | 3007/149872 [00:46<29:29, 82.98it/s]

0.9982690956453188 3.1418942638489082


  4%|▍         | 6009/149872 [01:24<41:20, 57.99it/s]

1.0677510946907498 3.3307134748056937


  6%|▌         | 9012/149872 [02:04<28:50, 81.41it/s]

...

 96%|█████████▌| 144013/149872 [30:51<01:11, 81.93it/s]

1.0188009104826914 3.18173401858883


 98%|█████████▊| 147012/149872 [31:29<00:33, 86.24it/s]

1.0198340240845767 3.1831901230418005


100%|██████████| 149872/149872 [32:06<00:00, 77.81it/s]


In [ ]:
100*incorrect_characters/total_nb_characters, 100*incorrect_words/total_nb_words

(1.020290707712771, 3.184495091188176)

In [ ]:
total_nb_words,total_nb_characters, incorrect_words, incorrect_characters

(1456055, 5942326, 46368, 60629)

In [ ]:
initial_total_nb_words,initial_total_nb_characters, initial_incorrect_words, initial_incorrect_characters = total_nb_words,total_nb_characters, incorrect_words, incorrect_characters

In [12]:
initial_total_nb_words,initial_total_nb_characters, initial_incorrect_words, initial_incorrect_characters = 1456055, 5942326, 46368, 60629

In [14]:
from tqdm import tqdm
total_nb_words= initial_total_nb_words
total_nb_characters = initial_total_nb_characters
incorrect_words=initial_incorrect_words
incorrect_characters=initial_incorrect_characters
for sample in tqdm(test_long_splitted):
    delta_incorrect_characters, delta_total_characters, delta_incorrect_words, delta_total_words = compute_error_numbers_for_sample(sample['input_ids'], sample['labels'], model, step=100, no_slide=True)
    total_nb_words += delta_total_words
    total_nb_characters += delta_total_characters
    incorrect_words += delta_incorrect_words
    incorrect_characters += delta_incorrect_characters
100*incorrect_characters/total_nb_characters, 100*incorrect_words/total_nb_words

100%|██████████| 262/262 [00:12<00:00, 20.81it/s]


(1.0189973381912523, 3.180193286281267)

In [ ]:
total_nb_words= initial_total_nb_words
total_nb_characters = initial_total_nb_characters
incorrect_words=initial_incorrect_words
incorrect_characters=initial_incorrect_characters
for sample in tqdm(test_long):
    delta_incorrect_characters, delta_total_characters, delta_incorrect_words, delta_total_words = compute_error_numbers_for_sample(sample['input_ids'], sample['labels'], model, step=100)
    total_nb_words += delta_total_words
    total_nb_characters += delta_total_characters
    incorrect_words += delta_incorrect_words
    incorrect_characters += delta_incorrect_characters
100*incorrect_characters/total_nb_characters, 100*incorrect_words/total_nb_words

100%|██████████| 128/128 [00:13<00:00,  9.32it/s]


(1.018381217305004, 3.1784266254808204)

In [ ]:
total_nb_words= initial_total_nb_words
total_nb_characters = initial_total_nb_characters
incorrect_words=initial_incorrect_words
incorrect_characters=initial_incorrect_characters
for sample in tqdm(test_long):
    delta_incorrect_characters, delta_total_characters, delta_incorrect_words, delta_total_words = compute_error_numbers_for_sample(sample['input_ids'], sample['labels'], model, step=40)
    total_nb_words += delta_total_words
    total_nb_characters += delta_total_characters
    incorrect_words += delta_incorrect_words
    incorrect_characters += delta_incorrect_characters
100*incorrect_characters/total_nb_characters, 100*incorrect_words/total_nb_words

100%|██████████| 128/128 [00:23<00:00,  5.48it/s]


(1.018264653894092, 3.1784945739731456)

In [ ]:
total_nb_words= initial_total_nb_words
total_nb_characters = initial_total_nb_characters
incorrect_words=initial_incorrect_words
incorrect_characters=initial_incorrect_characters
for sample in tqdm(test_long):
    delta_incorrect_characters, delta_total_characters, delta_incorrect_words, delta_total_words = compute_error_numbers_for_sample(sample['input_ids'], sample['labels'], model, step=10)
    total_nb_words += delta_total_words
    total_nb_characters += delta_total_characters
    incorrect_words += delta_incorrect_words
    incorrect_characters += delta_incorrect_characters
100*incorrect_characters/total_nb_characters, 100*incorrect_words/total_nb_words

100%|██████████| 128/128 [01:04<00:00,  1.97it/s]


(1.0183312615574704, 3.1784266254808204)

In [ ]:
total_nb_words= initial_total_nb_words
total_nb_characters = initial_total_nb_characters
incorrect_words=initial_incorrect_words
incorrect_characters=initial_incorrect_characters
for sample in tqdm(test_long):
    delta_incorrect_characters, delta_total_characters, delta_incorrect_words, delta_total_words = compute_error_numbers_for_sample(sample['input_ids'], sample['labels'], model, step=5)
    total_nb_words += delta_total_words
    total_nb_characters += delta_total_characters
    incorrect_words += delta_incorrect_words
    incorrect_characters += delta_incorrect_characters
100*incorrect_characters/total_nb_characters, 100*incorrect_words/total_nb_words

100%|██████████| 128/128 [01:46<00:00,  1.21it/s]


(1.0182813058099367, 3.1784266254808204)

In [ ]:
total_nb_words= initial_total_nb_words
total_nb_characters = initial_total_nb_characters
incorrect_words=initial_incorrect_words
incorrect_characters=initial_incorrect_characters
for sample in tqdm(test_long):
    delta_incorrect_characters, delta_total_characters, delta_incorrect_words, delta_total_words = compute_error_numbers_for_sample(sample['input_ids'], sample['labels'], model, step=1)
    total_nb_words += delta_total_words
    total_nb_characters += delta_total_characters
    incorrect_words += delta_incorrect_words
    incorrect_characters += delta_incorrect_characters
100*incorrect_characters/total_nb_characters, 100*incorrect_words/total_nb_words

100%|██████████| 128/128 [03:01<00:00,  1.42s/it]


(1.0184811288000712, 3.1790381619117443)